In [60]:
import os
import itk
import sys
import time
import ants
import json
# from itkwidgets import view --> This seems to cause the issue when an error happened, the kernel disconnects and stays in a zombie state
# import SimpleITK as sitk  # An alternative to itk
import glob
import shutil
import numpy as np
# import pydicom as pyd
# import dicom2nifti
from directory_tree import display_tree  # Nice tool to display directory trees (https://pypi.org/project/directory-tree/)

from datetime import timedelta

In [61]:
def list_folder_content(path, show_hidden=False):
    if show_hidden:
        ddfldrlst = os.listdir(path)
    else:
        ddfldrlst = list(filter(lambda item: not item.startswith('.'),os.listdir(path)))      
    return ddfldrlst

def display_folder_list(file_list):
    print('\n'.join(f'[{idx}] - {file_idx}' for idx, file_idx in enumerate(file_list)))

def get_path_to_process(full_path):
    print('Folder content:')
    print(display_tree(full_path, header=True, string_rep=True, show_hidden=False, max_depth=2))
    folder_content = list_folder_content(full_path)
    # Ideally we'll have only one sub-folder inside the PreTreatment folder. If more than one, then we have to choose, but by default, we'll select the first one.
    idx_reg = 0
    if len(folder_content) > 1:
        display_folder_list(folder_content)
        idx_sel = input(f'Select the folder with the dataset_to_process to process (0-{len(folder_content)-1} or just press Enter to proceed with sub-folder {folder_content[idx_reg]}):')
        if idx_sel:
            idx_reg = int(idx_sel)
    path2data = os.path.join(full_path, folder_content[idx_reg])
    print(f'Will process {folder_content[idx_reg]}')
    return path2data

def check_time_points(path_to_check, nmax = 6, verbose=False):
    if path_to_check is not None:
        nr_of_folders = list_folder_content(path_to_check)
        print(f'Folder {path_to_check} seems Ok' if len(nr_of_folders)== nmax else f'Error! Check path {path_to_check} is the correct one')
        if verbose:
            print('Listing folder content:')
            display_tree(path_to_check, max_depth=1)
        return nr_of_folders if len(nr_of_folders) == nmax else None
    else:
        return None

def add_prefix_to_filename(full_path, prefix=None):
    # Assume the last part of the path is the filename (with extension)
    file_path, file_name_ext = os.path.split(full_path)
    if prefix:
        updated_filename = '_'.join([prefix, file_name_ext])
        return os.path.join(file_path, updated_filename)
    else:
        return prefix
    
def itk2ants(itkInput):
    antsOutput = ants.from_numpy(itk.GetArrayFromImage(itkInput).T, 
                                 origin=tuple(itkInput.GetOrigin()), 
                                 spacing=tuple(itkInput.GetSpacing()), 
                                 direction=np.array(itkInput.GetDirection()))
    
    return antsOutput

def ants2itk(antsInput):
    itkOutput = itk.GetImageFromArray(antsInput.numpy().T)
    itkOutput.SetOrigin(antsInput.origin)
    itkOutput.SetSpacing(antsInput.spacing)
    itkOutput.SetDirection(antsInput.direction)

    return itkOutput

def getenv():
    """
    Requires sys and os modules:
    import sys
    import os
    Possible values for sys.platform are (https://docs.python.org/3/library/sys.html & https://stackoverflow.com/questions/446209/possible-values-from-sys-platform)
    ┍━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┑
    │  System             │ Value               │
    ┝━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━┥
    │ Linux               │ linux or linux2 (*) │
    │ Windows             │ win32               │
    │ Windows/Cygwin      │ cygwin              │
    │ Windows/MSYS2       │ msys                │
    │ Mac OS X            │ darwin              │
    │ OS/2                │ os2                 │
    │ OS/2 EMX            │ os2emx              │
    │ RiscOS              │ riscos              │
    │ AtheOS              │ atheos              │
    │ FreeBSD 7           │ freebsd7            │
    │ FreeBSD 8           │ freebsd8            │
    │ FreeBSD N           │ freebsdN            │
    │ OpenBSD 6           │ openbsd6            │
    │ AIX                 │ aix (**)            │
    ┕━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┙
    """
    if sys.platform == 'win32':
        env_home = 'HOMEPATH'
    elif (sys.platform == 'darwin') | (sys.platform == 'linux'):
        env_home = 'HOME'
    HOMEPATH = os.getenv(env_home)
    
    return HOMEPATH

def check_path_exist(path, file=False):
    """
    Flag FILE indicates the path contains a file name (FLAG=TRUE) or the path only points to a folder (FLAG=FALSE (Default))
    """
    if file:
        is_path = os.path.isfile(path)
    else:
        is_path = os.path.isdir(path)

    print(f'{"OK:" if is_path else "ERROR:"} Path to {"file" if file else "folder"} {path} does{"" if is_path else " NOT"} exist')

    return is_path

In [181]:
def register_inter_visit(fixed_volume_path, moving_volume_path, deformation_maps_path, config_path, method='elastix', elastix_config=None):
    if method.lower() == 'elastix':
        print('Elastix registration')
        par_set = set_registration_parameters(config_path, method, elastix_config)
        registration_maps_inter_visit = elastix_inter_visit_registration(fixed_volume_path, moving_volume_path, par_set, deformation_maps_path)
    elif method.lower() == 'ants':
        print('ANTs registration method')
        par_set = set_registration_parameters(config_path, method)
        registration_maps_inter_visit = ants_inter_visit_registration(fixed_volume_path, moving_volume_path, par_set, deformation_maps_path)
    else:
        registration_maps_inter_visit = None
        print(f'ERROR!: Registration method {method.upper()} unknown')

    return registration_maps_inter_visit

def set_registration_parameters(config_path, method='elastix', path_to_default_elastix=None):
    
    if method.lower() == 'ants':
        # ANTs
        # For details about possible values and description of parameters, see the help page: https://antspy.readthedocs.io/en/latest/registration.html
        # Default values (as listed in the hep page)
        par_set = {'type_of_transform': 'SyN', 
                   'initial_transform': None, 
                   'outprefix': '', 
                   'mask': None,
                   'moving_mask': None,
                   'mask_all_stages': False,
                   'grad_step': 0.2,
                   'flow_sigma': 3, 
                   'total_sigma': 0, 
                   'aff_metric': 'mattes', 
                   'aff_sampling': 32, 
                   'aff_random_sampling_rate': 0.2, 
                   'syn_metric': 'mattes', 
                   'syn_sampling': 32, 
                   'reg_iterations': (40, 20, 0),
                   'aff_iterations': (2100, 1200, 1200, 10), 
                   'aff_shrink_factors': (6, 4, 2, 1), 
                   'aff_smoothing_sigmas': (3, 2, 1, 0), 
                   'write_composite_transform': False, 
                   'random_seed': None
                   }
        # To ensure reproducibility of the results, set the random_seed to a constant value:
        par_set['random_seed'] = 42 #(just to keep along with the pop-culture reference, e.g. https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a
        # Save the parameters as a JSON file in the parameters folder:
        with open(f'{config_path}_ANTs.json', 'w') as fp:
            json.dump(par_set, fp)
    elif method.lower() == 'elastix':
        # Elastix
        print(f'Define the parameters for the registration. Please wait...')
        par_set = itk.ParameterObject.New()
        if path_to_default_elastix is None:
            print('ERROR: A path to configuration files must be provided')
            par_set = None
        
        for par_files in path_to_default_elastix:
            print(f'Adding parameters file {par_files}, please wait...')
            par_set.AddParameterFile(par_files)
            # Copy the parameters files to the output folder:
            print(f'Path to Config file: {config_path}_{os.path.split(par_files)[1]}')
            shutil.copy2(par_files, f'{config_path}_{os.path.split(par_files)[1]}')
    else:
        print(f"Registration algorithm {method.upper()} unknown")
        par_set = None

    print(f'Parameters set for {method}.upper() registration method:')
    print(par_set)
 
    return par_set


def elastix_inter_visit_registration(path_to_fixed, path_to_moving, parameters, path_to_output, debugmode=True):
    
    # path_to_output must be a two-elements dictionary:
    # path_to_output = {'path_to_deformation': '<root_path><deformation_name>.nii.gz'
    #                   'path_to_registered': '<root_path><aligned_volume>.nii.gz'}

    start_time = time.perf_counter()
    print(f'Loading fixed image from {path_to_fixed}')
    fixed_volume = itk.imread(path_to_fixed)
    if debugmode:
        time_to_read_fixed = time.perf_counter()
        print(f'************* Elapsed time to load Fixed Volume: {(time_to_read_fixed-start_time):0.2f}s')

    print(f'Loading fixed image from {path_to_moving}')
    moving_volume = itk.imread(path_to_moving)
    if debugmode:
        time_to_read_moving = time.perf_counter()
        print(f'************* Elapsed time to load Moving Volume: {(time_to_read_moving-time_to_read_fixed):0.2f}s')

    print('Apply histogram matching between fixed and moving images...')
    #moving_volume = itk.HistogramMatchingImageFilter(fixed_volume, moving_volume)
    moving_volume = itk.histogram_matching_image_filter(moving_volume, fixed_volume)
    if debugmode:
        time_to_histo_match = time.perf_counter()
        print(f'************* Elapsed time to run Histogram Matching (not implemented yet): {(time_to_histo_match-time_to_read_moving):0.2f}s')
   
    print(f'Registering {path_to_moving} to {path_to_fixed}, please wait...')
    warped_moving, result_transform = itk.elastix_registration_method(fixed_volume , 
                                                                      moving_volume,
                                                                      parameter_object=parameters,
                                                                      log_to_console=False)
    if debugmode:
        time_to_register = time.perf_counter()
        print(f'************* Elapsed time to run Registration: {(time_to_register - time_to_histo_match):0.2f}s')

    print(f'Deriving the deformation map (as an image), please wait...')
    deformation_field = itk.transformix_deformation_field(moving_volume, result_transform)
    if debugmode:
        time_to_deformation_map = time.perf_counter()
        print(f'************* Elapsed time to derive the deformation map: {(time_to_deformation_map - time_to_register):0.2f}s')

    print(f'Saving registered image and deformation maps, please wait...')
    registered_output = {'warpedmovout': warped_moving, 
                         'fwdtransforms': deformation_field,
                         'transformation': result_transform}
    print(f"saving registered volume at {path_to_output['path_to_registered']}")
    itk.imwrite(registered_output['warpedmovout'], path_to_output['path_to_registered'])
    if debugmode:
        time_to_save_registered_vol = time.perf_counter()
        print(f'************* Elapsed time to save the registered volume: {(time_to_save_registered_vol - time_to_deformation_map):0.2f}s')

    print(f"saving deformation map at {path_to_output['path_to_deformation']}")
    itk.imwrite(registered_output['fwdtransforms'], path_to_output['path_to_deformation'])
    if debugmode:
        time_to_save_def_map = time.perf_counter()
        print(f'************* Elapsed time to save the deformation map: {(time_to_save_def_map - time_to_save_registered_vol):0.2f}s')
    else:
        end_time = time.perf_counter()
        print(f'************* Elapsed time to run the whole registration process: {(end_time - start_time):0.2f}s')
    return registered_output

    
def ants_inter_visit_registration(path_to_fixed, path_to_moving, parameters, path_to_output, debugmode=True):
    
    start_time = time.perf_counter()
    fixed_volume = ants.image_read(path_to_fixed)
    if debugmode:
        time_to_read_fixed = time.perf_counter()
        print(f'************* Elapsed time to load the Fixed Volume: {(time_to_read_fixed-start_time):0.2f}s')

    moving_volume = ants.image_read(path_to_moving)
    if debugmode:
        time_to_read_moving = time.perf_counter()
        print(f'************* Elapsed time to load the Moving Volume: {(time_to_read_moving-time_to_read_fixed):0.2f}s')

    print('Apply histogram matching between fixed and moving images...')
    moving_volume = ants.histogram_match_image(moving_volume, fixed_volume)
    if debugmode:
        time_to_histo_match = time.perf_counter()
        print(f'************* Elapsed time to run Histogram Matching (not implemented yet): {(time_to_histo_match-time_to_read_moving):0.2f}s')

    registered_output = ants.registration(fixed=fixed_volume , 
                                          moving=moving_volume, 
                                          **parameters)
    if debugmode:
        time_to_register = time.perf_counter()
        print(f'************* Elapsed time to run the Registration: {(time_to_register - time_to_read_moving):0.2f}s')

    ants.image_write(registered_output['warpedmovout'], path_to_output['path_to_registered'])
    if debugmode:
        time_to_save_registered_vol = time.perf_counter()
        print(f'************* Elapsed time to save the registered volume: {(time_to_save_registered_vol - time_to_register):0.2f}s')

    # ants.image_write(registered_output['fwdtransforms'], path_to_output['path_to_deformation'])
    # Deformation map is provided as a list with path to the temporal file. so we can just copy that into the output:
    shutil.copy2(registered_output['fwdtransforms'][0], path_to_output['path_to_deformation'])
    if debugmode:
        time_to_save_def_map = time.perf_counter()
        print(f'************* Elapsed time to save (just transfer from temp folder) the deformation map: {(time_to_save_def_map - time_to_save_registered_vol):0.2f}s')
    else:
        end_time = time.perf_counter()
        print(f'************* Elapsed time to run the whole registration process: {(end_time - start_time):0.2f}s')

    return registered_output

def apply_transform(path_to_moving, path_to_transform):
    print('Placeholder to apply a transform to other images')

In [182]:
HOMEPATH = getenv()

SRCPATH = os.path.join(HOMEPATH, 'Data', 'fMRIBreastData')

TESTFLDR = 'tests'
DATAFLDR = 'datasets'
CFGSRCFLDR = 'configFiles'

studypath = os.path.join(SRCPATH, TESTFLDR)
configpath = os.path.join(SRCPATH, CFGSRCFLDR)
# Check the path exist and are correct:
check_path_exist(studypath)
check_path_exist(configpath)

OK: Path to folder /Users/joseulloa/Data/fMRIBreastData/tests does exist
OK: Path to folder /Users/joseulloa/Data/fMRIBreastData/configFiles does exist


True

In [183]:
DEBUGMODE = True
BATCHMODE = False # TRUE: it runs the registration for all dataset within STUDYPATH; FALSE (DEFAULT): allows to pick a specific dataset to register

What we do here, is for each patient data, we will register one of the post-treatment volumes (either the 1st Post-Contrast or Pre-contrast) to the pre-treatment equivalent.

As usual, I'll program it for a specific dataset, and then generalise to run for all datasets as a batch process

In [187]:
# For Elastix, use the default parameter set:
elastix_config = glob.glob(os.path.join(configpath, 'Par0032*.txt'))
fixed_volume_index = 2 # 1: Pre-Contrast; 2: 1st Post-contrast
registration_method = 'ants' # 'ants'

tests = sorted(list_folder_content(studypath))
if 'Test000' in tests:
    # This doesn't have to be processed, so must be removed:
    tests.remove('Test000')
print('Folders in the STUDY directory:')
display_folder_list(tests)

if not BATCHMODE:
    # # Pick up an option:
    test_idx = input(f'Pick up a valid index to select a test folder {tuple(range(len(tests)))} or type "a" to process all or "d" for default:')
    if test_idx == 'a':
        # Selecting all is equivalent to run in Batch mode
        BATCHMODE = True
        print(f'Changing the BATCHMODE flag to {BATCHMODE}')
    elif test_idx == 'd':
        if registration_method == 'elastix':
            test_idx = tests.index('Test011')
        elif registration_method == 'ants':
            test_idx = tests.index('Test012')
        else:
            test_idx = 0
        tests = [tests[test_idx]]
        print(f'Will process test folder {tests[0]}')
        if DEBUGMODE:
            print(f'Folder {tests[0]} content:')
            print(display_tree(os.path.join(studypath, tests[0], DATAFLDR), header=True, string_rep=True, show_hidden=False, max_depth=1))
    else:
        test_idx = int(test_idx)
        tests = [tests[test_idx]]
        print(f'Will process test folder {tests[0]}')
        if DEBUGMODE:
            print(f'Folder {tests[0]} content:')
            print(display_tree(os.path.join(studypath, tests[0], DATAFLDR), header=True, string_rep=True, show_hidden=False, max_depth=1))
else:
    print(f'Processing the whole data folder {studypath} as a batch process \n***Please be patient!!***')

Folders in the STUDY directory:
[0] - Test001
[1] - Test002
[2] - Test003
[3] - Test004
[4] - Test005
[5] - Test006
[6] - Test007
[7] - Test008
[8] - Test009
[9] - Test010
[10] - Test011
[11] - Test012
Will process test folder Test012
Folder Test012 content:
datasets/
├── CR-ANON68760/
├── DC-ANON97378/
├── EilB-ANON98269/
├── GL-ANON99397/
├── JB-ANON18218/
├── NE-ANON89073/
└── RICE00-RICE001/



In [188]:
# From the tests available, pickup a dataset to process:
if not BATCHMODE:
    test_to_process = tests[0]
    path_to_patients = os.path.join(studypath, test_to_process, DATAFLDR)
    patients = sorted(list_folder_content(path_to_patients))
    print(f'Folders in the {tests[0]} directory:')
    display_folder_list(patients)
    # # Pick up an option:
    patient_idx = input(f'Pick up a valid index to select a patient {tuple(range(len(patients)))} or type "a" to process all:')
    if patient_idx == 'a':
        print(f'Will process all patients in the test folder')
    else:
        patient_idx = int(patient_idx)
        patients = [patients[patient_idx]]
        print(f'Will process patient {patients[0]}')
else:
    print(f'Processing the whole data folder {studypath} as a batch process \n***Please be patient!!***')    


Folders in the Test012 directory:
[0] - CR-ANON68760
[1] - DC-ANON97378
[2] - EilB-ANON98269
[3] - GL-ANON99397
[4] - JB-ANON18218
[5] - NE-ANON89073
[6] - RICE00-RICE001
Will process patient CR-ANON68760


In [189]:
for test_to_process in tests:
    if BATCHMODE:
        path_to_patients = os.path.join(studypath, test_to_process, DATAFLDR)
        patients = sorted(list_folder_content(path_to_patients))
    for patient_to_process in patients:
        # Get the patientID from the initials:
        patientID = patient_to_process.split('-')[0]
        pre_treatment_pattern = '-'.join([patientID, 'Pre-Treatment'])
        post_treatment_pattern = '-'.join([patientID, 'Post-Treatment'])
        
        # Check there are two visits to register (if not, just skip):
        path_to_patient = os.path.join(path_to_patients, patient_to_process)
        visits = sorted(list_folder_content(path_to_patient))
        # Check the pre/post-treatment folder are present, both must exist to run the algorithm:
        # remember the general form:
        # if/else: [f(x) if condition else g(x) for x in sequence]
        # if only: [f(x) for x in sequence if condition]
        visits_fldr = [visit for visit in visits if (visit.startswith(pre_treatment_pattern) | visit.startswith(post_treatment_pattern))]
        if len(visits_fldr) == 2:
            proceed = True
            path_to_pretreatment = [os.path.join(path_to_patient, visit) for visit in visits_fldr if visit.startswith(pre_treatment_pattern)][0]
            path_to_postreatment = [os.path.join(path_to_patient, visit) for visit in visits_fldr if visit.startswith(post_treatment_pattern)][0]
            # inside a dataset, there is a numeric folder (actually, it must be the only folder in there) indicating the MRI series. In here, we take the fixed_volume_index from pre- and post-treatment to register each other:
            subdirs_list_pre = next(os.walk(path_to_pretreatment))[1]
            seq_nro_index_pre = 0
            if len(subdirs_list_pre) > 1:
                display_folder_list(subdirs_list_pre)
                seq_nro_index_pre = input(f'There are more than one subfolder, please pickup the correct one:')
            path_to_fixed_volume = glob.glob(os.path.join(path_to_pretreatment, subdirs_list_pre[seq_nro_index_pre], f'{fixed_volume_index}','*.nii.gz'))[0]
            print(f'Path to fixed Volume: {path_to_fixed_volume}')

            subdirs_list_post = next(os.walk(path_to_postreatment))[1]
            seq_nro_index_post = 0
            if len(subdirs_list_post) > 1:
                display_folder_list(subdirs_list_post)
                seq_nro_index_post = input(f'There are more than one subfolder, please pickup the correct one:')
            path_to_moving_volume = glob.glob(os.path.join(path_to_postreatment, subdirs_list_post[seq_nro_index_post], f'{fixed_volume_index}','*.nii.gz'))[0]
            print(f'Path to moving Volume: {path_to_moving_volume}')

            # Deformation map will be saved in the moving (i.e. post-treatment) folder with the name: deformation_map_<dataID>.nii.gz
            deformation_map_name = f'deformation_map_{registration_method}_fixedVol{fixed_volume_index:03d}_{os.path.split(path_to_postreatment)[1]}.nii.gz'
            registered_volume_name = f'inter_visit_aligned_{registration_method}_fixedVol{fixed_volume_index:03d}_{os.path.split(path_to_postreatment)[1]}.nii.gz'

            path_to_deformation_map = {'path_to_registered': os.path.join(path_to_postreatment, registered_volume_name),
                                       'path_to_deformation': os.path.join(path_to_postreatment, deformation_map_name)}
            # So we now have the path to fixed volume, moving, and output path to deformation map
            # these are passed into the registration function.
            # Additionally, the registration parameters will be stored in the parameters sub-folder for each test dataset:
            reg_cfg_path = os.path.join(studypath, test_to_process, 'parameters', f'{patient_to_process}_inter_visit_parameters')
            # If running elastix, the following must be passed into set_registration_parameters:
            # elastix_config
            start_time = time.perf_counter()
            print(f'Registering dataset {patient_to_process}, please wait...')
            register_output = register_inter_visit(path_to_fixed_volume, 
                                                   path_to_moving_volume, 
                                                   path_to_deformation_map, 
                                                   reg_cfg_path, 
                                                   method=registration_method, elastix_config=elastix_config)
            end_time = time.perf_counter()
            elp_time = end_time - start_time
            print(f'Ended registering dataset {patient_to_process}')
            print(f'Elapsed time: {elp_time:0.2f}[s]')
        else:
            proceed = False
            print(f'Skipping patient {path_to_patient}')
            continue


Path to fixed Volume: /Users/joseulloa/Data/fMRIBreastData/tests/Test012/datasets/CR-ANON68760/CR-Pre-Treatment-20221212/301/2/301_dyn_ethrive.nii.gz
Path to moving Volume: /Users/joseulloa/Data/fMRIBreastData/tests/Test012/datasets/CR-ANON68760/CR-Post-Treatment-20230120/301/2/301_dyn_ethrive.nii.gz
Registering dataset CR-ANON68760, please wait...
ANTs registration method
Parameters set for ants.upper() registration method:
{'type_of_transform': 'SyN', 'initial_transform': None, 'outprefix': '', 'mask': None, 'moving_mask': None, 'mask_all_stages': False, 'grad_step': 0.2, 'flow_sigma': 3, 'total_sigma': 0, 'aff_metric': 'mattes', 'aff_sampling': 32, 'aff_random_sampling_rate': 0.2, 'syn_metric': 'mattes', 'syn_sampling': 32, 'reg_iterations': (40, 20, 0), 'aff_iterations': (2100, 1200, 1200, 10), 'aff_shrink_factors': (6, 4, 2, 1), 'aff_smoothing_sigmas': (3, 2, 1, 0), 'write_composite_transform': False, 'random_seed': 42}
************* Elapsed time to load the Fixed Volume: 0.93s
**

In [140]:
fixed = ants.image_read(path_to_fixed_volume)
moving = ants.image_read(path_to_moving_volume)
parameters = set_registration_parameters(reg_cfg_path, method='ants')
registered_output = ants.registration(fixed=fixed,
                                      moving=moving,
                                      **parameters)


Parameters set for ants.upper() registration method:
{'type_of_transform': 'SyN', 'initial_transform': None, 'outprefix': '', 'mask': None, 'moving_mask': None, 'mask_all_stages': False, 'grad_step': 0.2, 'flow_sigma': 3, 'total_sigma': 0, 'aff_metric': 'mattes', 'aff_sampling': 32, 'aff_random_sampling_rate': 0.2, 'syn_metric': 'mattes', 'syn_sampling': 32, 'reg_iterations': (40, 20, 0), 'aff_iterations': (2100, 1200, 1200, 10), 'aff_shrink_factors': (6, 4, 2, 1), 'aff_smoothing_sigmas': (3, 2, 1, 0), 'write_composite_transform': False, 'random_seed': 42}
